In [1]:
import os
import configparser
from PIL import Image

import numpy as np

import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as datasets
from torchvision import transforms

from tqdm import tqdm

In [ ]:
# path to Git Repo from Google CoLab file
path = 'drive/Shareddrives/CSEN240_Group11/Facial-Recognition'

root_path = path if os.path.isdir(path) else ''

data_dir = os.path.join(root_path, 'training')

In [1]:
# load dataset
dataset = datasets.ImageFolder(data_dir, transform=lambda img: np.array(img).astype(float))

def get_mean_std(loader):
    # Compute the mean and standard deviation of all pixels in the dataset
    num_pixels = 0
    mean = 0.0
    std = 0.0
    images: torch.Tensor
    for images, _ in loader:
        batch_size, num_channels, height, width = images.shape
        num_pixels += batch_size * height * width
        mean += images.mean(axis=(0, 2, 3)).sum()
        std += images.std(axis=(0, 2, 3)).sum()

    mean /= num_pixels
    std /= num_pixels

    return mean, std

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

batch_size = 32
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
mean, std = get_mean_std(loader)

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

In [2]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(4 * 500 * 500, 128),
    nn.ReLU(),
    nn.Linear(128, 33)
)

In [ ]:
model = model.double()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    
    inputs: torch.Tensor
    for i, (inputs, labels) in enumerate(tqdm(loader)):
        inputs = inputs.permute(0, 3, 1, 2)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataset)
    print(f"Epoch {epoch+1} loss: {epoch_loss:.4f}")

In [ ]:
model_dst = os.path.join(root_path, 'models')
os.makedirs(model_dst, exist_ok=True)
torch.save(model, f'{model_dst}/nn.pth')
print('model saved successfully')

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = torch.load('models/nn.pth')
model.eval()

params = count_parameters(model)
print(params)

128023777


In [7]:
from tqdm import tqdm

# Define lists to store predictions and ground truth labels
predictions = []
ground_truth = []

# Iterate over the validation dataset
for inputs, labels in tqdm(loader):
    inputs = inputs.permute(0, 3, 1, 2)
    # Forward pass
    with torch.no_grad():  # No need to compute gradients during inference
        outputs = model(inputs)
    
    # Convert outputs to probabilities or class predictions depending on your model
    # For example, if your model is a classification model with softmax activation:
    probabilities = torch.softmax(outputs, dim=1)
    _, predicted_classes = torch.max(probabilities, 1)

    # Append predictions and ground truth labels to lists
    predictions.append(predicted_classes)
    ground_truth.append(labels)

100%|██████████| 332/332 [29:00<00:00,  5.24s/it]


In [23]:
from sklearn.metrics import accuracy_score

acc = []
for pred, gt in zip(predictions, ground_truth):
    p = pred.numpy()
    y = gt.numpy()

    accuracy = accuracy_score(p, y)
    acc.append(accuracy)

print(sum(acc)/len(acc))

0.03473268072289157
